In [7]:
import os
from multiprocessing import Pool, cpu_count


import numpy as np
import pandas as pd

import pickle
from tqdm.notebook import tqdm

import gc

In [8]:
path_to_market_data = os.getcwd() + "/BTCUSDT/"
path_to_market_data

'/home/roman/JupyterLab/otus_rl/project/data/BTCUSDT/'

In [9]:
dt_list = [x[:-4] for x in os.listdir(os.getcwd() + "/BTCUSDT/bid_ask")]
path_to_rl_data = "/home/roman/JupyterLab/otus_rl/project/rl_data"

In [10]:
dt_list = sorted(dt_list)
print(dt_list)

['2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-20', '2024-01-21']


In [11]:
def process_period(period):
    period_start, period_end = period
    subset = lt[(lt["tradeTime"] >= period_start) & (lt["tradeTime"] < period_end)].to_dict("records")
    return (period_start, period_end), subset

# Function to initialize the DataFrame in each worker process
def init_worker(lt_df):
    global lt
    lt = lt_df

In [13]:
for dt in tqdm(["2024-01-08"]):
    # read ba data
    ba = pd.read_csv(path_to_market_data + f"bid_ask/{dt}.csv")
    ba = ba.sort_values("update_id")

    # create periods from bid_ask
    period_starts = ba["received_time"][::50].values
    period_ends = period_starts[1:]
    
    period_starts_n = ba.index.to_list()[::50]
    period_ends_n = period_starts_n[1:]
    
    periods = list(zip(period_starts[:-1], period_ends))
    periods_n = list(zip(period_starts_n[:-1], period_ends_n))
    
    periods_map = dict(zip(periods_n, periods))
    # convert dataframe to list
    ba_list = ba.to_dict("records")
    del ba
    gc.collect()
    # store bid ask data in a dict
    ba_data_dict = {}
    for (period_start_n, period_end_n), (period_start, period_end) in periods_map.items():
        ba_data_dict[(period_start, period_end)] = ba_list[period_start_n:period_end_n]
    # save bid ask data
    with open(f"{path_to_rl_data}/bid_ask/{dt}.pickle", 'wb') as file:
        pickle.dump(ba_data_dict, file)
    del ba_data_dict
    gc.collect()
    # read trades data
    lt = pd.read_csv(path_to_market_data + f"last_trade/{dt}.csv")
    lt = lt.sort_values("tradeid")
    lt = lt.drop(["received_time", "event_time", "symbol", "buyerOrderId", "sellerOrderId", "ignore"], axis=1)
    # store trades data in a dict
    lt_df = lt
    with Pool(processes=18, initializer=init_worker, initargs=(lt_df,)) as pool:
        results = list(tqdm(pool.imap(process_period, periods), total=len(periods)))    
    lt_data_dict = dict(results)
    del lt
    gc.collect()
    # save trades data
    with open(f"{path_to_rl_data}/last_trade/{dt}.pickle", 'wb') as file:
        pickle.dump(lt_data_dict, file)
    del lt_data_dict
    gc.collect()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/132750 [00:00<?, ?it/s]

In [ ]:
for dt in tqdm_notebook(dt_list):
    ba = pd.read_csv(f"{path_to_market_data}/bid_ask/{dt}.csv")
    ba.drop(["symbol"], axis=1, inplace=True)
    ba = ba.sort_values("update_id")
    ba["received_time"] = pd.to_datetime(ba["received_time"])
    ba = ba.T.to_dict()

    bid_ask_rl_path = os.path.join(path_to_rl_data, "bid_ask")
    if not os.path.exists(bid_ask_rl_path):
        os.mkdir(bid_ask_rl_path)
    with open(os.path.join(bid_ask_rl_path, f"{dt}.pickle"), 'wb') as handle:
        pickle.dump(ba, handle, protocol=pickle.HIGHEST_PROTOCOL)

    lt = pd.read_csv(f"{path_to_market_data}/last_trade/{dt}.csv")
    lt.drop(["symbol", "buyerOrderId", "sellerOrderId", "ignore"], axis=1, inplace=True)
    lt["received_time"] = pd.to_datetime(lt["received_time"])
    lt["event_time"] = pd.to_datetime(lt["event_time"])
    lt["tradeTime"] = pd.to_datetime(lt["tradeTime"])
    lt = lt.sort_values("tradeid")
    lt = lt.T.to_dict()

    last_trade_rl_path = os.path.join(path_to_rl_data, "last_trade")
    if not os.path.exists(last_trade_rl_path):
        os.mkdir(last_trade_rl_path)
    with open(os.path.join(last_trade_rl_path, f"{dt}.pickle"), 'wb') as handle:
        pickle.dump(lt, handle, protocol=pickle.HIGHEST_PROTOCOL)

/tmp/ipykernel_3254416/3118045633.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for dt in tqdm_notebook(dt_list):


  0%|          | 0/14 [00:00<?, ?it/s]